*IMPORT NECESSARY DEPENDENCIES*

In [2]:
import cv2
import os
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from tensorflow import keras
import pathlib
from  sklearn import model_selection



In [3]:
path_to_data = f'C:/Users/user/Pictures/Saved Pictures/'

*CONVERT PATH TO A PATHLIB PATH FOR EASE OF MANIPULATION*

In [4]:
pathlib_wrapped = pathlib.Path(path_to_data)
pathlib_wrapped

WindowsPath('C:/Users/user/Pictures/Saved Pictures')

In [5]:
len(list(pathlib_wrapped.glob('Fj/*')))

113





*CREATING AN IMAGE DICTIONARY COMPRISING OF THE LABEL AND THE LIST OF ALL THE CORRESPONDING IMAGES*

In [6]:
images_dict = {'Dad': list(pathlib_wrapped.glob('Dad/*')), 
               'Fj' : list(pathlib_wrapped.glob('Fj/*')), 
               'Kwubeche' : list(pathlib_wrapped.glob('Kwubeche/*')),
               'Mum' : list(pathlib_wrapped.glob('Mum/*')), 
               'Shuu' : list(pathlib_wrapped.glob('Shuu/*')), 
               'Sinko' : list(pathlib_wrapped.glob('Sinko/*')),
               'TonyBlaze' : list(pathlib_wrapped.glob('TonyBlaze/*'))}









*CREATING THE LABEL DICTIONARY FOR ENCODING OF WORDS WITH NUMBERS AND VICE VERSA*

In [7]:
images_dict_label = {'Dad': 0, 
               'Fj' : 1, 
               'Kwubeche' : 2,
               'Mum' : 3, 
               'Shuu' : 4, 
               'Sinko' : 5,
               'TonyBlaze' : 6}





*PREPARE THE X AND Y VALUES FOR TRAINING*
GOING THROUGH ALL PICTURES, RESIZING EACH ONE AND UPDATING THE TRAINING LIST

In [8]:
X, y = [], []
for image_name, images in images_dict.items():
    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img, (224, 224))
        X.append(resized_img)
        y.append(images_dict_label[image_name])

In [9]:
X = np.array(X)
y = np.array(y)

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
X_train_scaled = X_train / 255
X_test_scaled = X_test / 255






*USING CNN TO TRAIN THE MODEL*


NUM_CLASSES IS THE NUMBER OF EXPECTED OUTPUTS

In [11]:
num_classes = 7

In [54]:

model = keras.Sequential([
    keras.layers.Conv2D(16, 3, padding= 'same', activation='relu'),
    keras.layers.MaxPooling2D(),
    
    keras.layers.Conv2D(32, 3, padding = 'same', activation='relu'),
    keras.layers.MaxPooling2D(),

    keras.layers.Conv2D(64, 3, padding = 'same', activation='relu'),
    keras.layers.MaxPooling2D(),
    
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(num_classes)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
              metrics=['accuracy'])
model.fit(X_train_scaled, y_train, epochs=20)

Epoch 1/20
13/13 [==============================] - 20s 1s/step - loss: 2.2882 - accuracy: 0.1995
Epoch 2/20
13/13 [==============================] - 16s 1s/step - loss: 1.7946 - accuracy: 0.3043
Epoch 3/20
13/13 [==============================] - 16s 1s/step - loss: 1.6077 - accuracy: 0.3913
Epoch 4/20
13/13 [==============================] - 16s 1s/step - loss: 1.2975 - accuracy: 0.5294
Epoch 5/20
13/13 [==============================] - 15s 1s/step - loss: 1.0623 - accuracy: 0.6240
Epoch 6/20
13/13 [==============================] - 16s 1s/step - loss: 0.7297 - accuracy: 0.7724
Epoch 7/20
13/13 [==============================] - 15s 1s/step - loss: 0.4770 - accuracy: 0.8491
Epoch 8/20
13/13 [==============================] - 15s 1s/step - loss: 0.3048 - accuracy: 0.9079
Epoch 9/20
13/13 [==============================] - 15s 1s/step - loss: 0.2029 - accuracy: 0.9386
Epoch 10/20
13/13 [==============================] - 15s 1s/step - loss: 0.1080 - accuracy: 0.9719
Epoch 11/20
13/13 [

In [55]:
model.evaluate(X_test_scaled,y_test)

5/5 [==============================] - 2s 317ms/step - loss: 3.1843 - accuracy: 0.5344


[3.1842892169952393, 0.5343511700630188]





USING DATA AUGMENTATION TECHNIQUE TO SOLVE THE OVERFITTING ISSUE WITH THE CNN MODEL


*INITIALIZING AN INSTANCE OF DATA AUGMENTATION USING RANDOM FLIP, RANDOM ROTATION AND RANDOM ZOOM TO CREATE MORE GENERIC INPUT IMAGES*

In [20]:
data_augmentation = keras.Sequential(
  [
    keras.layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                 input_shape=(224, 224, 3)),
    keras.layers.experimental.preprocessing.RandomRotation(0.1),
    keras.layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)

RETRAINING THE MODEL USING CNN AND DATA AUGMENTATION

In [57]:

model = keras.Sequential([
    data_augmentation,
    keras.layers.Conv2D(16, 3, padding= 'same', activation='relu'),
    keras.layers.MaxPooling2D(),
    
    keras.layers.Conv2D(32, 3, padding = 'same', activation='relu'),
    keras.layers.MaxPooling2D(),

    keras.layers.Conv2D(64, 3, padding = 'same', activation='relu'),
    keras.layers.MaxPooling2D(),
    keras.layers.Dropout(0.2),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(num_classes)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
              metrics=['accuracy'])
model.fit(X_train_scaled, y_train, epochs=20)

Epoch 1/30


13/13 [==============================] - 26s 1s/step - loss: 2.1517 - accuracy: 0.1944
Epoch 2/30
13/13 [==============================] - 22s 2s/step - loss: 1.8733 - accuracy: 0.2379
Epoch 3/30
13/13 [==============================] - 21s 2s/step - loss: 1.7432 - accuracy: 0.3197
Epoch 4/30
13/13 [==============================] - 20s 1s/step - loss: 1.6925 - accuracy: 0.3223
Epoch 5/30
13/13 [==============================] - 18s 1s/step - loss: 1.5297 - accuracy: 0.4015
Epoch 6/30
13/13 [==============================] - 23s 2s/step - loss: 1.4339 - accuracy: 0.4501
Epoch 7/30
13/13 [==============================] - 23s 2s/step - loss: 1.3807 - accuracy: 0.4731
Epoch 8/30
13/13 [==============================] - 21s 2s/step - loss: 1.3764 - accuracy: 0.5090
Epoch 9/30
13/13 [==============================] - 23s 2s/step - loss: 1.1932 - accuracy: 0.5857
Epoch 10/30
13/13 [==============================] - 22s 2s/step - loss: 1.2423 - accuracy: 0.5371
Epoch 11/30
13/13 [===========

In [58]:
model.evaluate(X_test_scaled,y_test)

5/5 [==============================] - 3s 408ms/step - loss: 4.0107 - accuracy: 0.3435


[4.0106611251831055, 0.3435114622116089]

In [31]:
len(X_test_scaled), len(X_train)

(59, 175)

USING A PRETRAINED MODEL FROM TENSORFLOW HUB TO CUSTOM RETRAIN THE MODEL FOR BETTER PERFORMANCE

In [12]:
import tensorflow_hub as hub

IMAGE_SHAPE = (224, 224)


In [17]:
feature_extractor_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

pretrained_model_without_top_layer = hub.KerasLayer(
    feature_extractor_model, input_shape=(224, 224, 3), trainable=False)

In [21]:


model = tf.keras.Sequential([
  data_augmentation,
  pretrained_model_without_top_layer,
  tf.keras.layers.Dense(num_classes)
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 keras_layer (KerasLayer)    (None, 1280)              2257984   
                                                                 
 dense (Dense)               (None, 7)                 8967      
                                                                 
Total params: 2,266,951
Trainable params: 8,967
Non-trainable params: 2,257,984
_________________________________________________________________


TRAINING USING TRANSFER LEARNING

In [22]:
model.compile(
  optimizer="adam",
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['acc'])

model.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50


13/13 [==============================] - 30s 1s/step - loss: 2.1319 - acc: 0.2302
Epoch 2/50
13/13 [==============================] - 13s 1s/step - loss: 1.6574 - acc: 0.3862
Epoch 3/50
13/13 [==============================] - 14s 1s/step - loss: 1.3186 - acc: 0.5627
Epoch 4/50
13/13 [==============================] - 13s 1s/step - loss: 1.1935 - acc: 0.6240
Epoch 5/50
13/13 [==============================] - 16s 1s/step - loss: 1.0296 - acc: 0.6650
Epoch 6/50
13/13 [==============================] - 13s 1s/step - loss: 0.9452 - acc: 0.7084
Epoch 7/50
13/13 [==============================] - 14s 1s/step - loss: 0.8605 - acc: 0.7494
Epoch 8/50
13/13 [==============================] - 13s 1s/step - loss: 0.7779 - acc: 0.7775
Epoch 9/50
13/13 [==============================] - 13s 1s/step - loss: 0.7344 - acc: 0.8031
Epoch 10/50
13/13 [==============================] - 14s 1s/step - loss: 0.6591 - acc: 0.8184
Epoch 11/50
13/13 [==============================] - 13s 1s/step - loss: 0.6452 

In [23]:
model.evaluate(X_test_scaled,y_test)

5/5 [==============================] - 5s 798ms/step - loss: 1.0938 - acc: 0.6565


[1.0937598943710327, 0.6564885377883911]

In [24]:
predict_path = 'C:/Users/user/Pictures/test_img/Agumma.jpg'
predict_img = cv2.imread(predict_path)
pred_img = cv2.resize(predict_img, (224, 224))
pred_img.shape

(224, 224, 3)

In [25]:
pred_img = np.array(pred_img)
pred_img = pred_img/255

# Expand dimensions to create batch size of 1
pred_img = np.expand_dims(pred_img, axis=0)

# Print the shape of the preprocessed image
print(pred_img.shape)

(1, 224, 224, 3)


In [26]:
prediction = model.predict(pred_img)
prediction = np.argmax(prediction)
prediction


1/1 [==============================] - 2s 2s/step


3

In [27]:
images_dict_label2 = { 0 : 'Dad', 
               1:'Fj', 
               2:'Kwubeche',
               3:'Mum', 
               4:'Shuu', 
               5:'Sinko',
               6:'TonyBlaze'}

In [28]:

print('This should be', images_dict_label2[prediction])

This should be Mum


In [29]:
import pickle
with open('family_img_classification2.pickle','wb') as f:
    pickle.dump(model,f)

INFO:tensorflow:Assets written to: ram://549f4519-4bc7-4a8b-92a4-be01d2ce5aa0/assets


INFO:tensorflow:Assets written to: ram://549f4519-4bc7-4a8b-92a4-be01d2ce5aa0/assets


In [ ]:
path = f'C:/Users/user/Desktop/Python_lessons/ML/ML/Portfolio/face_detection/family_img_classification3.h5'

model.save(path)

In [26]:
import json

with open("images_dict_label.json","w") as f:
    f.write(json.dumps(images_dict_label))